In [1]:
import numpy as np

from E2_AnalysisHelpers import *

rnds = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19",
        "20", "21", "22", "23", "24", "25"]

print("done")

done


In [2]:
frames = []

for yr in yrs:
    print(yr)
    for lge in lges:
        for rnd in rnds:
            preFiles = getFiles(csvFinalDir, f"{yr}-{lge}-Rnd_{rnd}-PreRace.csv")
            resFiles = getFiles(csvFinalDir, f"{yr}-{lge}-Rnd_{rnd}-Result.csv")
            
            if len(resFiles) > 0:
                resFile = resFiles[-1]
                if len(resFiles) > 1:
                    print("")
                    for i in resFiles: print(i)
                    print("")
                    print(resFile)
                    print("")
            if len(preFiles) > 0:
                preFile = preFiles[0]

                preFrame = pd.read_csv(preFile)

                if len(resFiles) > 0:
                    resFrame = pd.read_csv(resFile)
                    sessions = resFrame.session.unique()
                    finRac = sessions[-1]
                    resSmolFrame = resFrame.loc[(resFrame["session"] == finRac)]
                    sessions = resSmolFrame.session.unique()
                    if len(sessions) > 1:
                        print(f"\n{yr} {lge} rnd_{rnd}")
                        print(sessions)
                                                
                    rdrFrame = resSmolFrame[["rdr_num", "pos"]]
                    rdrFrame = rdrFrame.drop_duplicates()
                    rdrFrame = rdrFrame.rename(columns = {"pos": "result"})
                    nFrame = preFrame.join(rdrFrame.set_index("rdr_num"), on = "rdr_num")
                    val = {"result": "DNF"}
                    nF = nFrame.fillna(value = val)
                    
                    try:
                        nF["start_pos"] = nF["start_pos"].astype(int)
                    except:
                        pass
                    
                else:
                    preFrame["result"] = "none"
                    nF = preFrame

                frames.append(nF)

wF = pd.concat(frames)
print("\ndone")

2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998

done


In [26]:
weatherFiles = getFiles(csvWeatherDir, "*.csv")
weatherFrames = []

for file in weatherFiles:
    weatherFrame = pd.read_csv(file)
    weatherFrames.append(weatherFrame)

weatherFrame = pd.concat(weatherFrames)
weatherFrame.drop(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13"], axis = 1, inplace = True)
weatherFrame = weatherFrame[weatherFrame["Year"].notna()]
weatherFrame["Year"] = weatherFrame["Year"].astype(int)
weatherFrame.columns = weatherFrame.columns.str.lower()

dateData = weatherFrame["date"].str.split(",", n = 2, expand = True)
weatherFrame["month"] = dateData[0]
weatherFrame["day"] = dateData[1]
weatherFrame["yr"] = dateData[2]
weatherFrame.drop(columns =["date", "track", "year"], inplace = True)

weatherFrame = weatherFrame[["month", "day", "yr", "league", "session_type", "track_conditions", "track_temp", "air_temp", "humidity"]]
weatherFrame = weatherFrame.rename(columns={"league": "lge", "session_type": "session", "track_conditions": "conditions"})
weatherFrame["day"] = weatherFrame["day"].astype(int)

weatherFrame.head()

,month,day,yr,lge,session,conditions,track_temp,air_temp,humidity
0,March,26,2021,MotoGP,FP1,Dry,26º,28º,54%
1,March,26,2021,MotoGP,FP2,Dry,26º,25º,70%
2,March,27,2021,MotoGP,FP3,Dry,49º,37º,19%
3,March,27,2021,MotoGP,FP4,Dry,31º,27º,51%
4,March,27,2021,MotoGP,Q1,Dry,30º,27º,45%


In [27]:
weatherFrame["yr"] = weatherFrame["yr"].astype(int)
weatherFrame["day"] = weatherFrame["day"].astype(int)

wF["yr"] = wF["yr"].astype(int)
wF["day"] = wF["day"].astype(int)

wholeFrame = pd.merge(wF, weatherFrame, how = "left",
                      left_on = ["month", "day", "yr", "lge", "session"],
                      right_on = ["month", "day", "yr", "lge", "session"])

wholeFrame.head()

,index,month,day,yr,lge,rnd,session,trk,pos,rdr_num,...,one_scaled,two_scaled,thr_scaled,fr_scaled,start_pos,result,conditions,track_temp,air_temp,humidity
0,2021-GP-1-FP1-1000,March,26,2021,MotoGP,1,FP1,Losail International Circuit,1,21,...,0.939528,0.813348,0.877293,0.871884,7.0,18,Dry,26º,28º,54%
1,2021-GP-1-FP1-1001,March,26,2021,MotoGP,1,FP1,Losail International Circuit,1,21,...,0.574489,0.709908,0.822517,0.843218,7.0,18,Dry,26º,28º,54%
2,2021-GP-1-FP1-1002,March,26,2021,MotoGP,1,FP1,Losail International Circuit,1,21,...,0.563690,0.704039,0.810042,0.838111,7.0,18,Dry,26º,28º,54%
3,2021-GP-1-FP1-1003,March,26,2021,MotoGP,1,FP1,Losail International Circuit,1,21,...,0.560450,0.709096,0.816195,0.837927,7.0,18,Dry,26º,28º,54%
4,2021-GP-1-FP1-1004,March,26,2021,MotoGP,1,FP1,Losail International Circuit,1,21,...,0.556550,0.697056,0.809560,0.833452,7.0,18,Dry,26º,28º,54%


In [31]:
# replace this "Style de Aragon" junk
# print(wholeFrame.trk.unique())
# wholeFrame.loc[wholeFrame["trk"] == Style de Aragon, "trk"] = "Aragón"

wholeFrame.loc[wholeFrame["trk"] == "Style de Aragon"]

,index,month,day,yr,lge,rnd,session,trk,pos,rdr_num,...,one_scaled,two_scaled,thr_scaled,fr_scaled,start_pos,result,conditions,track_temp,air_temp,humidity
1100292,2010-GP-13-FP1-1000,September,17,2010,MotoGP,13,FP1,Style de Aragon,1,26,...,0.120202,0.872019,0.441383,0.678184,3.0,2,NaN,NaN,NaN,NaN
1100293,2010-GP-13-FP1-1001,September,17,2010,MotoGP,13,FP1,Style de Aragon,1,26,...,0.067533,0.770318,0.412889,0.634482,3.0,2,NaN,NaN,NaN,NaN
1100294,2010-GP-13-FP1-1002,September,17,2010,MotoGP,13,FP1,Style de Aragon,1,26,...,0.062898,0.741058,0.392426,0.611542,3.0,2,NaN,NaN,NaN,NaN
1100295,2010-GP-13-FP1-1003,September,17,2010,MotoGP,13,FP1,Style de Aragon,1,26,...,0.060826,0.729614,0.390685,0.604632,3.0,2,NaN,NaN,NaN,NaN
1100296,2010-GP-13-FP1-1004,September,17,2010,MotoGP,13,FP1,Style de Aragon,1,26,...,0.059906,0.719307,0.386948,0.599552,3.0,2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167790,2010-125cc-13-QP-1466,September,18,2010,125cc,13,QP,Style de Aragon,31,86,...,0.036493,0.618313,0.348022,0.494050,30.0,DNF,NaN,NaN,NaN,NaN
1167791,2010-125cc-13-QP-1467,September,18,2010,125cc,13,QP,Style de Aragon,31,86,...,0.034712,0.601415,0.345480,0.490515,30.0,DNF,NaN,NaN,NaN,NaN
1167792,2010-125cc-13-QP-1468,September,18,2010,125cc,13,QP,Style de Aragon,31,86,...,0.034926,0.610712,0.345490,0.489728,30.0,DNF,NaN,NaN,NaN,NaN
1167793,2010-125cc-13-QP-1469,September,18,2010,125cc,13,QP,Style de Aragon,31,86,...,0.035062,0.610848,0.349072,0.488706,30.0,DNF,NaN,NaN,NaN,NaN


50
50
Index(['index', 'month', 'day', 'yr', 'lge', 'rnd', 'session', 'trk', 'pos',
       'rdr_num', 'f_name', 'l_name', 'nat', 'team', 'manu', 'num_of_laps',
       'run_num', 'f_tire', 'r_tire', 'laps_on_f', 'laps_on_r', 'lap_num',
       'lap_time', 'lap_seconds', 'lap_val', 'pit', 'sec_one', 'one_seconds',
       'one_val', 'sec_two', 'two_seconds', 'two_val', 'sec_thr',
       'thr_seconds', 'thr_val', 'sec_four', 'four_seconds', 'four_val',
       'avg_spd', 'lap_scaled', 'one_scaled', 'two_scaled', 'thr_scaled',
       'fr_scaled', 'start_pos', 'result', 'conditions', 'track_temp',
       'air_temp', 'humidity'],
      dtype='object')


In [21]:
print(wholeFrame.trk.unique())

['Losail' 'Algarve' 'Jerez' 'Le Mans' 'Mugello' 'Catalunya' 'Sachsenring'
 'Assen' 'Red Bull Ring' 'Silverstone' 'Aragón' 'Misano' 'COTA' 'Brno'
 'Valencia' 'Argentina' 'Chang' 'Motegi' 'Phillip Island' 'Sepang'
 'Indianapolis' 'Laguna Seca' 'Estoril' 'Style de Aragon' 'Donington Park'
 'Shanghai' 'Istanbul']


In [ ]:
# wF.loc[(wF["start_pos"] == )]
# re.loc[(resFrame["session"] == finRac)]

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)

# nF = nFrame.drop(["run_num", "f_tire", "r_tire", "laps_on_f", "laps_on_r", "pit", "lge", "rnd", "trk", "manu", "index",
#                   "month", "day", "yr", "pos", "nat", "num_of_laps", "lap_num", "lap_time", "lap_val", "sec_one", "one_val",
#                   "sec_two", "two_val", "sec_thr", "thr_val", "sec_four", "four_val", "avg_spd"], axis = 1)

# val = {"result": "DNF"}
# xf = nF.fillna(value = val)
# xf = xf[["session", "f_name", "l_name", "rdr_num", "team", "lap_seconds", "lap_scaled", "one_seconds", "one_scaled",
#          "two_seconds", "two_scaled", "thr_seconds", "thr_scaled", "four_seconds", "fr_scaled", "start_pos", "result"]]

# yf = xf.sort_values(by=["session", "result"])

# yf